<center>

<h1 style="text-align:center"> (Side) Effects </h1>
</center>

## Why Side Effects

* We have only used **purely functional** feature of OCaml
* Our study of lambda calculus used only **purely functional** features

* Not quite true!
  + We have used `print_endline`, `printf` and other features to display our results to screen.
  + We have used `fresh` function to implement substitution in Lambda Calculus.
* It is sometimes useful to write programs that have **side effects**

## Side effects

Side effects include

* Mutating (i.e., destructively updating) the values of program state. 
* Reading from standard input, printing to standard output.
* Reading and writing to files, sockets, pipes etc.
* ...
* Composing, sending and receiving emails, editing documents, writing this slide, etc.

## Side effects in OCaml

* OCaml programs can include side effects
* Features
  + Mutations: Reference cells, Arrays, Mutable record fields
  + I/O of all sorts
* In this lecture, **Mutations**

$
\require{color}
\newcommand{\cpurple}[1]{{\color{\purple}{\text{#1}}}}
\newcommand{\cred}[1]{{\color{\red}{\text{#1}}}}
$

## Reference cells

* Aka "refs" or "ref cell"
* Essentially a pointer to a location in memory
* The binding of a variable to a ref cell is $\cpurple{immutable}$ 
  + but the contents of the ref cell may $\cred{change}$.

## Reference cells

Allocate new ref cells with `ref`

In [1]:
let r = ref 1

val r : int ref = {contents = 1}


Update using `:=` and read using `!`.

In [4]:
r := !r + 1

- : unit = ()


In [5]:
!r

- : int = 4


## Reference Cells: Types

In [1]:
ref

- : 'a -> 'a ref = <fun>


In [2]:
(!)

- : 'a ref -> 'a = <fun>


In [3]:
(:=)

- : 'a ref -> 'a -> unit = <fun>


## Sequencing using `;`

We can chain together expressions with side effects using the semi-colon operator

In [6]:
r := !r + 1;
!r

- : int = 5


This is equivalent to

In [7]:
let _ = r := !r + 1 in
!r

- : int = 6


## Implementing a counter

In [1]:
let make_counter init =
  let c = ref init in
  fun () -> 
    (c := !c + 1; !c)

val make_counter : int -> unit -> int = <fun>


In [2]:
let next = make_counter 0

val next : unit -> int = <fun>


In [3]:
next ()

- : int = 1


In [4]:
next ()

- : int = 2


Aside: A function of the form `fun () -> ...` is also called a **thunk**.

## Side effects make reasoning hard

* **Referential transparency** allows replacing $e$ with $v$ if $e =_{\beta} v$.
* Side effects break referential transparency.

## Referential transparency

Consider the function `foo`:

In [7]:
let foo x = x + 1

val foo : int -> int = <fun>


In [8]:
let baz = foo 10

val baz : int = 11


`baz` can now be optimised to

In [ ]:
let baz = 11

## Referential transparency

Consider the function `bar`:

In [10]:
let next = make_counter 0
let bar x = x + next()

val next : unit -> int = <fun>


val bar : int -> int = <fun>


In [11]:
let qux = bar 10

val qux : int = 11


Can we now optimise `qux` to:

In [12]:
let qux = 11

val qux : int = 11


**NO!** Referential transparency breaks under side effects. 

## Example: Implementing pointers using `ref`

Desired properties:
* Can be `null`
* Points to a location whose value can be changed

In [1]:
type 'a pointer = 'a ref option

type 'a pointer = 'a ref option


In [2]:
let null: 'a pointer = None

val null : 'a pointer = None


In [4]:
let malloc (v : 'a) : 'a pointer = Some (ref v)

val malloc : 'a -> 'a pointer = <fun>


In [5]:
let p = malloc 42

val p : int pointer = Some {contents = 42}


## Example: Implementing pointers using `ref`

In [6]:
exception Segfault

let deref (p : 'a pointer) : 'a = 
  match p with
  | None -> raise Segfault
  | Some r -> !r

exception Segfault


val deref : 'a pointer -> 'a = <fun>


In [7]:
deref p

- : int = 42


In [8]:
deref null

error: runtime_error

In [9]:
let assign (p : 'a pointer) (v: 'a) : unit = 
  match p with 
  | None -> raise Segfault
  | Some r -> r := v

val assign : 'a pointer -> 'a -> unit = <fun>


In [10]:
assign p 2;
deref p

- : int = 2


## Example 2: Recursive functions without `rec` keyword

In [11]:
let rec fact n = if n = 0 then 1 else n * fact (n-1)

val fact : int -> int = <fun>


In [12]:
fact 5

- : int = 120


Let's implement `fact` without using `rec` but with function references!

## Example 2: Recursive functions without `rec` keyword

Step 1: Declare a function reference to a dummy function. Need to ensure that the type matches the function to be implemented.

In [13]:
let fact0 = ref (fun x -> x + 0)

val fact0 : (int -> int) ref = {contents = <fun>}


Step 2: Redefine the original function without `rec` keyword, replacing the recursive call with the function reference.

In [14]:
let fact_norec n = if n = 0 then 1 else n * !fact0 (n-1)

val fact_norec : int -> int = <fun>


## Example 2: Recursive functions without `rec` keyword

Step 3: Assign the function reference to the newly defined function

In [6]:
fact0 := fact_norec

- : unit = ()


In [7]:
fact_norec 5

- : int = 120


Also called "tying the recursive knot".

## Aliases

References may create aliases.

What is the result of this program?

In [13]:
let x = ref 10 in
let y = ref 10 in
let z = x in 
z := !z + 1;
!x + !y

- : int = 21


<center>
    
<img src="heap2.svg" width="350">
</center>

* Variables `z` and `x` are said to be **aliases**
  + They refer to the same object in the program heap.

## Equality

* The `=` that we have been using is known as **structural equality**
  + Checks whether the values are structurally equal. 
  + `[1;2;3] = [1;2;3]` evaluates to `true`. 
* Because of references, one may also want to ask whether two expressions are **aliases**
  + This equality is known as **physical equality**.
  + OCaml uses `==` to check for physical equality.

## Equality : Types

In [11]:
(=)

- : 'a -> 'a -> bool = <fun>


In [12]:
(==)

- : 'a -> 'a -> bool = <fun>


## Equality : Quiz
```ocaml
let l1 = [1;2;3];; 
let l2 = l1;; 
let l3 = [1;2;3];;
let r1 = ref l1;;
let r2 = r1;;
let r3 = ref l3;;
```
which of the following are true?

(1)`l1 = l2` (2)`l1 == l2` (3) `l1 = l3` (4) `l1 == l3`  (5) `l1 = [1;2;3]`

(6) `l1 == [1;2;3]` (7) `r1 == r2`  (8) `r1 = r2` (9) `r1 == r3` (10) `r1 = r3`

## Equality : Quiz

In [5]:
let l1 = [1;2;3];; 
let l2 = l1;; 
let l3 = [1;2;3];;
let r1 = ref l1;;
let r2 = r1;;
let r3 = ref l3;;

val l1 : int list = [1; 2; 3]


val l2 : int list = [1; 2; 3]


val l3 : int list = [1; 2; 3]


val r1 : int list ref = {contents = [1; 2; 3]}


val r2 : int list ref = {contents = [1; 2; 3]}


val r3 : int list ref = {contents = [1; 2; 3]}


## Equality

Which of the following are true?

(1)`l1 = l2` (2)`l1 == l2` (3) `l1 = l3` (4) `l1 == l3`  (5) `l1 = [1;2;3]`

(6) `l1 == [1;2;3]` (7) `r1 == r2`  (8) `r1 = r2` (9) `r1 == r3` (10) `r1 = r3`

References are structurally equal iff their contents are structurally equal.

In [6]:
l1 == l2

- : bool = true


## Mutable Record Fields

Ref cells are essentially syntactic sugar over **mutable record fields**.

```ocaml
type 'a ref = { mutable contents: 'a }
let ref x = { contents = x }
let ( ! ) r = r.contents
let ( := ) r newval = r.contents <- newval
```

* This type is declared in `Pervasives`


## Doubly-linked list

In [8]:
(* The type of elements *)
type 'a node = { 
  v : 'a;
  mutable next : 'a node option;
  mutable prev : 'a node option
}

(* The type of list *)
type 'a dllist = 'a node option ref

type 'a node = {
  v : 'a;
  mutable next : 'a node option;
  mutable prev : 'a node option;
}


type 'a dllist = 'a node option ref


## Double-linked list

In [9]:
let create () : 'a dllist = ref None
let first (t : 'a dllist) = !t
let is_empty (t : 'a dllist) = !t = None

let v node = node.v
let next node = node.next
let prev node = node.prev

val create : unit -> 'a dllist = <fun>


val first : 'a dllist -> 'a node option = <fun>


val is_empty : 'a dllist -> bool = <fun>


val v : 'a node -> 'a = <fun>


val next : 'a node -> 'a node option = <fun>


val prev : 'a node -> 'a node option = <fun>


## Doubly-linked list

In [14]:
(* [insert_first t v] inserts a new node [n] with value [v] 
   as the first node in the list [t]. Returns [n]. *)
let insert_first (t : 'a dllist) v =
  let n = { prev = None; next = !t; v = v} in
  begin match !t with
  | Some old_first -> old_first.prev <- Some n
  | None -> ()
  end;
  t := Some n;
  n

val insert_first : 'a dllist -> 'a -> 'a node = <fun>


## Doubly-linked list

In [15]:
(* [insert_after n v] inserts a new node [n'] whose value 
   is [v] after the node [n]. Returns [n']. *)
let insert_after n v =
  let n' = { v = v; prev = Some n; next = n.next } in
  begin match n.next with
  | Some old_next -> old_next.prev <- Some n'
  | None -> ()
  end;
  n.next <- Some n';
  n'

val insert_after : 'a node -> 'a -> 'a node = <fun>


## Doubly-linked list

In [16]:
(* [remove t n] removes the node [n] from the list [t] *)
let remove (t:'a dllist) n =
  begin match n.prev with
  | Some prev -> prev.next <- n.next
  | None -> t := n.next
  end;
  begin match n.next with
  | Some next -> next.prev <- n.prev;
  | None -> ()
  end;
  n.prev <- None;
  n.next <- None

val remove : 'a dllist -> 'a node -> unit = <fun>


## Doubly-linked list

In [17]:
(* [iter t f] applies [f] on each value in the list 
   from list to right *)
let iter (t : 'a dllist) (f : 'a -> unit) =
  let rec loop l = match l with
    | None -> ()
    | Some el -> f (v el); loop (next el)
  in
  loop !t

val iter : 'a dllist -> ('a -> unit) -> unit = <fun>


## Doubly-linked list

In [18]:
let l = create ();;
let n0 = insert_first l 0;;
let n1 = insert_after n0 1;;
insert_after n1 2

val l : '_weak2 dllist = {contents = None}


val n0 : int node = {v = 0; next = None; prev = None}


val n1 : int node =
  {v = 1; next = None; prev = Some {v = 0; next = Some <cycle>; prev = None}}


- : int node =
{v = 2; next = None;
 prev =
  Some
   {v = 1; next = Some <cycle>;
    prev = Some {v = 0; next = Some <cycle>; prev = None}}}


## Doubly-linked list

In [ ]:
iter l (Printf.printf "%d\n%!")

## Arrays

Collection type with efficient random access.

In [30]:
let a = [| 1;2;3 |]

val a : int array = [|1; 2; 3|]


Array elements are retrieved using `a.(i)` syntax where `a` is the array and `i` is the index.

In [31]:
a.(0)

- : int = 1


Arrays bounds are checked at access.

In [32]:
a.(3)

error: runtime_error

## Arrays

Array can be updated with `<-` syntax.

In [33]:
a.(1) <- 0; 
a

- : int array = [|1; 0; 3|]


In [34]:
a.(1)

- : int = 0


* Arrays behave very similar to arrays in C
 + See [Array module](https://caml.inria.fr/pub/docs/manual-ocaml/libref/Array.html) documentation in the OCaml standard library

## Benefits of immutability

* Programmer doesn’t have to think about aliasing
    + can concentrate on other aspects of code
* Language implementation is free to share objects on the heap, which is cheap
* Often easier to reason about whether code is correct
* Perfect fit for concurrent programming

But
* Some data structures (hash tables, arrays, ...) are more efficient if imperative

Recommendation

* Use immutable data structures unless performance can't be compromised